In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/output/analytical.csv", low_memory=False)
df.head()

,casenumber,incident_date,death_date,age,gender,race,latino,cold_related,heat_related,commissioner_district,...,toxic_primary,toxic_secondary,tramadol_primary,tramadol_secondary,u-47700_primary,u-49900_primary,valeryl_primary,xylazine_primary,xylazine_secondary,nitazene
0,ME2022-06565,2022-07-12T02:38:00.000,2022-07-12,74.0,Male,White,0,0,0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,ME2022-06564,2022-07-11T23:46:00.000,2022-07-12,NaN,Male,Black,0,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,ME2022-06563,2022-07-02T00:00:00.000,2022-07-11,28.0,Male,Black,0,0,0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,ME2022-06562,2022-07-11T20:48:00.000,2022-07-11,62.0,Female,White,0,0,0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,ME2022-06561,2022-07-11T19:50:00.000,2022-07-11,62.0,Male,White,0,0,0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
df['death_date'] = pd.to_datetime(df['death_date'])
total_records = df.shape[0]
df = df[df['death_date'] < pd.to_datetime('07-01-2022')]
print(f"{total_records - df.shape[0]} records removed")

284 records removed


In [4]:
control1 = df[(df['fentanyl_primary'] == 1) | (df['fentanyl_secondary'] == 1)].copy()
case = control1[(control1['xylazine_primary'] == 1) | (control1['xylazine_secondary'] == 1)].copy()
control2 = df[
    ((df['eth_alc_primary'] == 1) | (df['eth_alc_secondary'] == 1))
    & ((df['opiate_primary'] != 1) | (df['opiate_secondary'] != 1))
].copy()

In [6]:
def label(row) -> str:
    if row['fentanyl_primary'] == 1 or row['fentanyl_secondary'] == 1:
        # fentanyl related
        if row['xylazine_primary'] == 1 or row['xylazine_secondary'] == 1:
            # xylazine case
            return 'xylazine'
        else:
            # fentanyl control
            return 'fentanyl'
    elif (row['eth_alc_primary'] == 1 or row['eth_alc_secondary'] == 1) and (row['opiate_primary'] != 1 or row['opiate_secondary'] != 1):
        # alcohol control
        return 'alcohol'
    else:
        return 'other'


In [11]:
df['group'] = df.apply(lambda row:
    label(row),
    axis=1
)
no_other = df[df['group'] != 'other'].copy()
no_other.groupby('group').size()

group
alcohol     1693
fentanyl    6661
xylazine     265
dtype: int64

In [12]:
no_other.shape

(8619, 240)

In [13]:
no_other.to_csv("combined.csv", index=False)